In [ ]:
!pip install ipywidgets plotly scikit-learn

import string
import numpy as np
import pandas as pd
import warnings
import random
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

warnings.filterwarnings('ignore')

# Load and preprocess data
file_path = '/content/netflix_data.csv'
data = pd.read_csv(file_path)

# Pre-Processing of Data
class Cleantext:
    def separate_text(self, text):
        unique = set()
        for t in str(text).split(','):
            unique.add(t.strip().lower())
        return ' '.join(unique)

    def remove_space(self, text):
        return str(text).replace(' ', '').lower()

    def remove_punc(self, text):
        text = str(text).lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        return ' '.join(text.split())

# Fill missing values with empty strings
data.fillna('', inplace=True)

# Cleaning the data
cleaner = Cleantext()
data['director'] = data['director'].apply(cleaner.separate_text)
data['cast'] = data['cast'].apply(cleaner.separate_text)
data['listed_in'] = data['listed_in'].apply(cleaner.separate_text)
data['description'] = data['description'].apply(cleaner.remove_punc)

# Convert 'release_year' to integer
data['release_year'] = pd.to_numeric(data['release_year'], errors='coerce').fillna(0).astype(int)

# Ensure 'imdb_rating' is numeric
data['imdb_rating'] = pd.to_numeric(data['imdb_rating'], errors='coerce')

# Concatenate text columns to create a single text representation for each movie
def preprocess_text(row):
    return f"{row['description']} {row['type']} {row['director']} {row['cast']} {row['listed_in']} {row['rating']}"

data['combined_text'] = data.apply(preprocess_text, axis=1)

# Model Development
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['combined_text'])

# Use NearestNeighbors for optimization
n_neighbors = min(50, len(data))  # Use 50 or the number of items if less than 50
nn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)

# Optimized Recommendation System
def get_recommendations(title, df=data, top_n=50):
    idx = df.index[df['title'].str.contains(title, case=False, na=False)].tolist()
    if not idx:
        return pd.DataFrame()  # Return empty DataFrame if title not found
    idx = idx[0]

    distances, indices = nn_model.kneighbors(tfidf_matrix[idx].reshape(1, -1), n_neighbors=top_n+1)

    similar_movies_indices = indices.flatten()[1:]  # Exclude the movie itself
    similar_movies_distances = distances.flatten()[1:]

    similar_movies = df.iloc[similar_movies_indices]
    similar_movies['similarity'] = 1 - similar_movies_distances  # Convert distance to similarity

    return similar_movies.sort_values('similarity', ascending=False)

# UI Components
title_input = widgets.Text(description='Movie/Show:', style={'description_width': 'initial'})
imdb_range_slider = widgets.FloatRangeSlider(
    value=[0, 10],
    min=0,
    max=10.0,
    step=0.1,
    description='IMDB Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
year_input = widgets.IntText(description='Year:', style={'description_width': 'initial'})
director_input = widgets.Text(description='Director:', style={'description_width': 'initial'})
genre_input = widgets.Text(description='Genre:', style={'description_width': 'initial'})
cast_input = widgets.Text(description='Cast:', style={'description_width': 'initial'})
recommend_button = widgets.Button(description='Get Recommendations', button_style='primary')
output = widgets.Output()

# Trivia game function
def play_trivia(movie):
    questions = [
        f"What year was '{movie['title']}' released?",
        f"Who directed '{movie['title']}'?",
        f"What genre does '{movie['title']}' belong to?",
        f"Can you name one of the cast members of '{movie['title']}'?",
    ]
    answers = [
        str(movie['release_year']),
        movie['director'],
        movie['listed_in'],
        movie['cast']
    ]

    score = 0
    question_widget = widgets.HTML()
    answer_widget = widgets.Text(description="Your answer:")
    submit_button = widgets.Button(description="Submit")
    result_widget = widgets.HTML()

    display(question_widget, answer_widget, submit_button, result_widget)

    current_question = 0

    def check_answer(b):
        nonlocal score, current_question
        user_answer = answer_widget.value.lower()
        correct_answer = answers[current_question].lower()

        if user_answer in correct_answer:
            result_widget.value = "<p style='color: green;'>Correct!</p>"
            score += 1
        else:
            result_widget.value = f"<p style='color: red;'>Sorry, the correct answer was: {answers[current_question]}</p>"

        current_question += 1
        if current_question < len(questions):
            question_widget.value = f"<h3>Question {current_question + 1}:</h3><p>{questions[current_question]}</p>"
            answer_widget.value = ""
        else:
            question_widget.value = f"<h3>Game Over!</h3><p>You scored {score} out of {len(questions)}!</p>"
            answer_widget.layout.display = 'none'
            submit_button.layout.display = 'none'

    submit_button.on_click(check_answer)
    question_widget.value = f"<h3>Question 1:</h3><p>{questions[0]}</p>"

filtered_recommendations = None

def on_button_clicked(b):
    global filtered_recommendations
    with output:
        clear_output()
        title = title_input.value
        min_imdb, max_imdb = imdb_range_slider.value
        year = year_input.value
        director = director_input.value.lower()
        genre = genre_input.value.lower()
        cast = cast_input.value.lower()

        recommendations = get_recommendations(title)

        if recommendations.empty:
            print(f"No recommendations found for {title}. Please try another title.")
            return

        # Apply filters
        filtered_recommendations = recommendations.copy()

        filtered_recommendations = filtered_recommendations[
            (filtered_recommendations['imdb_rating'] >= min_imdb) &
            (filtered_recommendations['imdb_rating'] <= max_imdb)
        ]

        if year:
            filtered_recommendations = filtered_recommendations[filtered_recommendations['release_year'] == year]
        if director:
            filtered_recommendations = filtered_recommendations[filtered_recommendations['director'].str.contains(director, case=False, na=False)]
        if genre:
            filtered_recommendations = filtered_recommendations[filtered_recommendations['listed_in'].str.contains(genre, case=False, na=False)]
        if cast:
            filtered_recommendations = filtered_recommendations[filtered_recommendations['cast'].str.contains(cast, case=False, na=False)]

        if filtered_recommendations.empty:
            print("No recommendations match the specified criteria. Please try adjusting your filters.")
            return

        print("\nRecommended Movies and TV Shows:")
        for i, (_, item) in enumerate(filtered_recommendations.iterrows(), 1):
            print(f"{i}. {item['title']} (Type: {item['type']}, IMDB: {item['imdb_rating']:.1f}, Year: {item['release_year']})")
            print(f"   Director: {item['director']}")
            print(f"   Genre: {item['listed_in']}")
            print(f"   Cast: {item['cast']}")
            print(f"   Description: {item['description']}\n")

        # Trivia game
        play_trivia_button = widgets.Button(description="Play Trivia Game", button_style='success')
        display(play_trivia_button)

        def on_play_trivia_clicked(b):
            clear_output()
            movie_choice = widgets.Dropdown(
                options=[(item['title'], i) for i, (_, item) in enumerate(filtered_recommendations.iterrows())],
                description='Choose a movie/show:',
                style={'description_width': 'initial'}
            )
            start_game_button = widgets.Button(description="Start Game", button_style='info')
            display(movie_choice, start_game_button)

            def on_start_game(b):
                chosen_movie = filtered_recommendations.iloc[movie_choice.value]
                clear_output()
                play_trivia(chosen_movie)

            start_game_button.on_click(on_start_game)

        play_trivia_button.on_click(on_play_trivia_clicked)

recommend_button.on_click(on_button_clicked)

# Main Function
def main():
    display(widgets.VBox([
        title_input,
        imdb_range_slider,
        year_input,
        director_input,
        genre_input,
        cast_input,
        recommend_button,
        output
    ]))

if __name__ == "__main__":
    main()